In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW


In [32]:
# Load dataset
file_path = '/content/BenchmarkUddinSO-ConsoliatedAspectSentiment.xls'  # Update this to your file path
data = pd.read_excel(file_path)
df = data[['sent', 'ManualLabel', 'codes']]  # Adjust column names as per your dataset

# Filter for performance related entries and prepare labels for binary classification
df_performance = df[df['codes'].str.contains('Documentation', case=False)]
df_performance['ManualLabel'] = df_performance['ManualLabel'].apply(lambda x: 1 if x == 'p' else 0)

# Split the dataset into 60% training and 40% testing
train, test = train_test_split(df_performance, test_size=0.4, random_state=42)

<ipython-input-32-b71753d3e359>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_performance['ManualLabel'] = df_performance['ManualLabel'].apply(lambda x: 1 if x == 'p' else 0)


In [33]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train['sent'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test['sent'].tolist(), truncation=True, padding=True, max_length=512)

train_dataset = SentimentDataset(train_encodings, train['ManualLabel'].tolist())
test_dataset = SentimentDataset(test_encodings, test['ManualLabel'].tolist())


In [34]:
!pip install transformers[torch]

In [35]:
!pip install accelerate -U

In [36]:
import torch
from torch.utils.data import DataLoader
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score

# Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.5f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Average Loss: 0.62612
Epoch 2/3, Average Loss: 0.60854
Epoch 3/3, Average Loss: 0.58368


In [37]:
import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Set the model to evaluation mode
model.eval()
predictions, true_labels = [], []
total_inference_time = 0
total_samples = 0

# Ensure that 'no_grad' is used to prevent updating the model during inference
with torch.no_grad():
    for i, batch in enumerate(test_loader):  # corrected here
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        print(f"Processing batch {i+1}/{len(test_loader)}...")  # Log current batch number

        start_time = time.time()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        inference_time = time.time() - start_time

        total_inference_time += inference_time
        total_samples += input_ids.size(0)

        logits = outputs.logits
        preds = logits.argmax(dim=1).cpu().numpy()
        labels = batch['labels'].cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels)

        if i == 3:  # for instance, break after 3 batches
            break

# Compute metrics
overall_accuracy = accuracy_score(true_labels, predictions)
overall_recall = recall_score(true_labels, predictions, average='macro')
overall_precision = precision_score(true_labels, predictions, average='macro')
overall_f1_score = f1_score(true_labels, predictions, average='macro')

print(f'Overall Accuracy: {overall_accuracy:.4f}')
print(f'Overall Recall: {overall_recall:.4f}')
print(f'Overall Precision: {overall_precision:.4f}')
print(f'Overall F1 Score: {overall_f1_score:.4f}')

# Calculate total and average inference times
total_time_seconds = total_inference_time
sample_latency_seconds = total_inference_time / total_samples
print(f'Total Time (seconds): {total_time_seconds:.6f} seconds')
print(f'Samples Processed: {total_samples}')
print(f'Latency Per Sample (seconds): {sample_latency_seconds:.6f} seconds')


Processing batch 1/13...
Processing batch 2/13...
Processing batch 3/13...
Processing batch 4/13...
Overall Accuracy: 0.7500
Overall Recall: 0.5000
Overall Precision: 0.3750
Overall F1 Score: 0.4286
Total Time (seconds): 36.690585 seconds
Samples Processed: 32
Latency Per Sample (seconds): 1.146581 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
